In [2]:
import os
import sys
%matplotlib inline

import numpy as np
import pandas as pd
import ipywidgets
from scipy.optimize import Bounds
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from src import data
from src import efficient_frontier
from src import plot
from src import portfolio
from src import capm
from src import backtesting
import numpy as np
import pandas as pd
import ipywidgets
from scipy.optimize import Bounds
import datetime
from dateutil import rrule
import yfinance as yf
import warnings

warnings.filterwarnings("ignore", category=FutureWarning, module='pandas')

In [3]:
x = pd.Timestamp(np.datetime64('2000-01-01'))
x1 = pd.Timestamp(np.datetime64('2022-01-01'))

In [10]:
weights = np.array([1/3,1/3,1/3])
min_esg_score = 1400
max_esg_score = 2000
df = pd.read_excel('../data/ESG_DATA_S&P500.xlsx')
dates = [x,x1]

start_year = dates[0]
end_year = dates[1]

Bounds1 = Bounds(-1,2) #How willing we are to go short and to invest in one particular stock
Wanted_return = 0.20 #Only used when using the wanted_return constraint
maximum_risk = 0.10 #Either used when using maximum risk constraint or cmle portfolio
sharpe_type = "No_extra_constraint"  # rename to constraint, can be either Wanted_return or Maximum_risk or No_extra_constraint
risk_free_rate = 0.01
covariance_window = 5 #has to be in years
rebalancing_freq = 'monthly'

strategy1 = {'df': df,
 'weights': weights,
 'min_esg_score': min_esg_score,
 'max_esg_score': max_esg_score,
 'Bounds1': Bounds1,
 'sharpe_type': sharpe_type,
 'x': x,
 'x1': x1,
 'Wanted_return': Wanted_return,
 'maximum_risk': maximum_risk,
 'rebalancing_freq': rebalancing_freq,
 'risk_free_rate': risk_free_rate,
 'start_year' : start_year,
 'end_year' : end_year}


In [7]:
backtesting.backtesting(strategy1, 'yearly', 1, datetime.datetime(2015,1,1),datetime.datetime(2022,1,1),10,'SPY')

Min. Risk = 6.446% => Return: 1.227%  Sharpe Ratio = 0.19
Max. Sharpe Ratio = 0.24 => Return: 2.00%  Risk: 8.236%
Excpected return on investment is 113.60178227102489%
Min. Risk = 6.104% => Return: 1.438%  Sharpe Ratio = 0.24
Max. Sharpe Ratio = 0.29 => Return: 2.18%  Risk: 7.512%
Excpected return on investment is 112.25961588768081%
Min. Risk = 6.848% => Return: 1.472%  Sharpe Ratio = 0.21
Max. Sharpe Ratio = 0.27 => Return: 2.35%  Risk: 8.655%
Excpected return on investment is 111.25944225874889%
Min. Risk = 6.600% => Return: 1.698%  Sharpe Ratio = 0.26
Max. Sharpe Ratio = 0.30 => Return: 2.29%  Risk: 7.666%
Excpected return on investment is 112.7257729066124%
Min. Risk = 5.959% => Return: 1.915%  Sharpe Ratio = 0.32
Max. Sharpe Ratio = 0.37 => Return: 2.55%  Risk: 6.867%
Excpected return on investment is 108.70606342775405%
Min. Risk = 5.090% => Return: 2.063%  Sharpe Ratio = 0.41
Max. Sharpe Ratio = 0.42 => Return: 2.24%  Risk: 5.298%
Excpected return on investment is 110.662386512

([       AMZN      MSFT
  0  0.776073  0.223927,
         AMZN      MSFT
  0  0.716068  0.283932,
         AMZN      MSFT
  0  0.671318  0.328682,
         AMZN      MSFT
  0  0.736839  0.263161,
         AMZN      MSFT
  0  0.556882  0.443118,
         AMZN      MSFT
  0  0.644408  0.355592,
         AMZN      MSFT
  0  0.448815  0.551185,
         AMZN      MSFT
  0  0.450249  0.549751],
 [   ESG_score_of_portfolio
  0             1488.684146,
     ESG_score_of_portfolio
  0             1504.945638,
     ESG_score_of_portfolio
  0             1517.072953,
     ESG_score_of_portfolio
  0             1499.316541,
     ESG_score_of_portfolio
  0             1548.084984,
     ESG_score_of_portfolio
  0             1524.365442,
     ESG_score_of_portfolio
  0             1577.371163,
     ESG_score_of_portfolio
  0             1576.982641],
 [[1.214207851337865],
  [1.3312875320596818],
  [1.15541014490206],
  [1.3045029339608936],
  [1.4561468760164218],
  [1.8874160270498312],
  [1.9501

In [ ]:
#len(data5)
#print(data5[7],data5[6],data5[5])

plot.compare_returns1(data5[7],data5[5],data5[6])